In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
from numpy import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from IPython import display


#tf.compat.v1.disable_eager_execution()
%matplotlib inline

# Import data set and transform to tensor of right dimension

In [2]:
#Import both Samples
df_1 = pd.read_csv("data_sample1.csv")
#df_2= pd.read_csv("data_sample2.csv")


Dieser Teil ist da um das zweite Sample in Trainings und Testdaten aufzuteilen ohne Customer_Journeys aufzuteilen:

Das ist nur temporär um mit weniger Daten (schneller) zu arbeiten, während Code noch nicht ganz steht

In [3]:
df_train= df_1[df_1['journey_id'].isin(range(2000))]
df_test = df_1[df_1['journey_id'].isin(range(2000,3000,1))]
df_valid = df_1[df_1['journey_id'].isin(range(3000,4000,1))]
#print(df_train)
#print(df_test)
#print(df_valid)

Function that transforms the dataset into a tensor of the right form

In [4]:
def mta2tensor(df):

    groups = df.groupby('journey_id')
    df_transaction = df['transaction']
    df = df.drop('transaction', axis=1)

    first = 1
    for i in df['journey_id'].unique():

        x1 = groups.get_group(i)
        x1 = x1.drop(['journey_id'], axis=1)
        x1 = x1.to_numpy()
        #print('x1:',x1)
        y1 = df_transaction.loc[groups.get_group(i).index]
        y1 = y1.to_numpy()
        #print('y1:',y1)

        if first == 1:
            x_ten = tf.ragged.constant([x1])
            y_ten = tf.ragged.constant([y1])
            first = 0
        else:
            x_ten = tf.concat((x_ten, [x1]), axis=0)
            y_ten = tf.concat((y_ten, [y1]), axis=0)
    return x_ten, y_ten


In [5]:
X_train, Y_train = mta2tensor(df_train)
print('Train set done')
X_test, Y_test = mta2tensor(df_test)
print('Test set done')
X_valid, Y_valid = mta2tensor(df_valid)
print('Validation set done')


x1: [[2.00000000e+00 2.60000000e+01 8.00000000e+00 3.78217375e+01]
 [2.00000000e+00 2.60000000e+01 2.00000000e+00 3.71675542e+02]
 [2.00000000e+00 2.60000000e+01 2.00000000e+00 3.72569926e+02]
 ...
 [2.00000000e+00 2.60000000e+01 3.00000000e+00 6.33904268e+03]
 [2.00000000e+00 2.60000000e+01 3.00000000e+00 6.34876494e+03]
 [2.00000000e+00 2.60000000e+01 3.00000000e+00 6.35476648e+03]]
x1: [[1.00000000e+00 2.60000000e+01 1.00000000e+00 4.76498242e+01]
 [1.00000000e+00 2.60000000e+01 1.00000000e+00 1.44504576e+02]
 [1.00000000e+00 2.60000000e+01 1.00000000e+00 2.31590877e+02]
 [1.00000000e+00 2.60000000e+01 1.00000000e+00 5.75003114e+03]]
x1: [[2.00000000e+00 7.00000000e+00 1.00000000e+00 7.61855597e+01]
 [2.00000000e+00 7.00000000e+00 1.00000000e+00 3.71711574e+03]
 [2.00000000e+00 7.00000000e+00 1.00000000e+00 3.71714652e+03]
 [2.00000000e+00 7.00000000e+00 1.00000000e+00 3.71715774e+03]
 [2.00000000e+00 0.00000000e+00 1.00000000e+00 5.06389050e+03]]
x1: [[1.00000000e+00 3.50000000e+01

In [6]:
print(X_train) #nur zur Überprüfung obs passt

<tf.RaggedTensor [[[2.0, 26.0, 8.0, 37.8217374999658],
  [2.0, 26.0, 2.0, 371.675541944394],
  [2.0, 26.0, 2.0, 372.569926388853],
  ...,
  [2.0, 26.0, 3.0, 6339.04268111102],
  [2.0, 26.0, 3.0, 6348.76494027785],
  [2.0, 26.0, 3.0, 6354.76647583337]],
 [[1.0, 26.0, 1.0, 47.6498241666705],
  [1.0, 26.0, 1.0, 144.504576111096],
  [1.0, 26.0, 1.0, 231.590876666654],
  [1.0, 26.0, 1.0, 5750.03114472225]], [[2.0, 7.0, 1.0, 76.1855597221875],
                                        [2.0, 7.0, 1.0, 3717.11574305559],
                                        [2.0, 7.0, 1.0, 3717.14652416663],
                                        [2.0, 7.0, 1.0, 3717.15773555561],
                                        [2.0, 0.0, 1.0, 5063.89049583336]],
 [[1.0, 35.0, 1.0, 131.709395555605],
  [1.0, 35.0, 1.0, 131.743337777792],
  [1.0, 35.0, 2.0, 260.60566083336],
  [1.0, 5.0, 1.0, 1126.17355138896],
  [1.0, 36.0, 5.0, 4194.44635027781],
  [1.0, 36.0, 2.0, 4626.06707250001],
  [1.0, 36.0, 1.0, 4626.2758477

In [ ]:
#ein erster Test (inzwischen unrelevant)

#i=9
#y1 = df_transaction.loc[groups.get_group(i).index]
#y1 = y1.to_numpy()
#x1 = groups.get_group(i)
#x1 = x1.drop(['journey_id'], axis=1)
#x1 = x1.to_numpy()
#print(y1)
#print(x1)

#i=7
#y2 = df_transaction.loc[groups.get_group(i).index]
#y2 = y2.to_numpy()
#x2 = groups.get_group(i)
#x2 = x2.drop(['journey_id'], axis=1)
#x2 = x2.to_numpy()
#print(y2)
#print(x2)



In [4]:
#ein Test (inzwischen unrelevant)

#cj1 = np.random.rand(2,3)
#print('cj1:', cj1)
#cj2 = np.random.rand(4,3)
#print('cj2:', cj2)
#cj3 = 2*np.random.rand(3,3)
#print('cj3:', cj3)

cj1: [[0.34890166 0.73308812 0.11514617]
 [0.06148234 0.99534152 0.24609195]]
cj2: [[0.9513413  0.89278808 0.28177263]
 [0.72809586 0.97347739 0.1177666 ]
 [0.98040375 0.46649105 0.01780298]
 [0.92424585 0.65054003 0.76977565]]
cj3: [[1.13168662 1.20194488 1.26265602]
 [0.48060021 1.53675102 0.80302834]
 [1.98795681 1.86155067 1.22901019]]


In [25]:
#ein  Test (inzwischen unrelevant)

#tens  = np.stack((cj1,cj2),axis=0)
#print(tens)
#tensor = tf.ragged.constant([cj1])
#tf.print(tensor)
#tensor = tf.concat(([cj1],[cj2]), axis=0)
#tensor
#cj1 = np.random.rand(2,3)
#cj3 = 2*np.random.rand(3,3)

#tensor = tf.ragged.constant([cj1])
#tensor = tf.concat([tensor,[cj3]], axis=0)
#print(tensor)

<tf.RaggedTensor [[[0.2699899387960891, 0.9444304935479712, 0.040734933260874406],
  [0.17981373405747347, 0.1961031284465491, 0.9890948412340083]] ,
 [[1.747859694284373, 0.736116818122577, 0.9407473305395304],
  [1.5843158309550045, 1.1199106611144243, 1.8297503733990965],
  [0.8614319016260144, 1.9005111820033977, 0.3756037906519336]]]>


In [8]:
#ein Test (inzwischen unrelevant)

#x = tf.constant([[[ 1,  2,  3],
#                  [ 4,  5,  6]],
#                  [[ 7,  8,  9],
#                  [10, 11, 12]]])
#tf.transpose(x, perm=[1, 0, 2])


<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 7,  8,  9]],

       [[ 4,  5,  6],
        [10, 11, 12]]])>

In [23]:
#Bestimmung der Batch_size
a=tf.shape(X_train)[0]
int(a)

99

Alles ist irrelevant, weil ragged Tensoren für weitere Sachen nicht nutzbar sind

# Start with GRU Model

In [9]:
import tensorflow as tf

class GRU:
    """Implementation of a Gated Recurrent Unit (GRU) as described in [1].

    [1] Chung, J., Gulcehre, C., Cho, K., & Bengio, Y. (2014). Empirical evaluation of gated recurrent neural networks on sequence modeling. arXiv preprint arXiv:1412.3555.  [Titel anhand dieser ArXiv-ID in Citavi-Projekt übernehmen]

    Arguments
    ---------
    input_dimensions: int
        The size of the input vectors (x_t).
    hidden_size: int
        The size of the hidden layer vectors (h_t).
    dtype: obj
        The datatype used for the variables and constants (optional).
    """

    def __init__(self, input_dimensions, hidden_size, dtype=tf.float64):
        self.input_dimensions = input_dimensions
        self.hidden_size = hidden_size

        # Weights for input vectors of shape (input_dimensions, hidden_size)
        self.Wr = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wr')
        self.Wz = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wz')
        self.Wh = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wh')

        # Weights for hidden vectors of shape (hidden_size, hidden_size)
        self.Ur = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Ur')
        self.Uz = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uz')
        self.Uh = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uh')

        # Biases for hidden vectors of shape (hidden_size,)
        self.br = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='br')
        self.bz = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bz')
        self.bh = tf.Variable(tf.random.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bh')

        # Define the input layer placeholder
        self.input_layer = tf.compat.v1.placeholder(dtype=tf.float64, shape=(None, None, input_dimensions), name='input')

        # Put the time-dimension upfront for the scan operator
        self.x_t = tf.transpose(self.input_layer, [1, 0, 2], name='x_t')

        # A little hack (to obtain the same shape as the input matrix) to define the initial hidden state h_0
        self.h_0 = tf.matmul(self.x_t[0, :, :], tf.zeros(dtype=tf.float64, shape=(input_dimensions, hidden_size)), name='h_0')

        # Perform the scan operator
        self.h_t_transposed = tf.scan(self.forward_pass, self.x_t, initializer=self.h_0, name='h_t_transposed')

        # Transpose the result back
        self.h_t = tf.transpose(self.h_t_transposed, [1, 0, 2], name='h_t')

    def forward_pass(self, h_tm1, x_t):
        """Perform a forward pass.

        Arguments
        ---------
        h_tm1: np.matrix
            The hidden state at the previous timestep (h_{t-1}).
        x_t: np.matrix
            The input vector.
        """
        # Definitions of z_t and r_t
        z_t = tf.sigmoid(tf.matmul(x_t, self.Wz) + tf.matmul(h_tm1, self.Uz) + self.bz)
        r_t = tf.sigmoid(tf.matmul(x_t, self.Wr) + tf.matmul(h_tm1, self.Ur) + self.br)

        # Definition of h~_t
        h_proposal = tf.tanh(tf.matmul(x_t, self.Wh) + tf.matmul(tf.multiply(r_t, h_tm1), self.Uh) + self.bh)

        # Compute the next hidden state
        h_t = tf.multiply(1 - z_t, h_tm1) + tf.multiply(z_t, h_proposal)

        return h_t


In [25]:
a = tf.shape(X_train)[0]
batch_size = int(a)
# The input has 2 dimensions: dimension 0 is reserved for the first term and dimension 1 is reserved for the second term
input_dimensions = 4

# Arbitrary number for the size of the hidden state
hidden_size = 16

tf.compat.v1.disable_eager_execution()
# Initialize a session
session = tf.compat.v1.Session()

# Create a new instance of the GRU model
gru = GRU(input_dimensions, hidden_size)

# Add an additional layer on top of each of the hidden state outputs
W_output = tf.Variable(tf.random.truncated_normal(dtype=tf.float64, shape=(hidden_size, 1), mean=0, stddev=0.01))
b_output = tf.Variable(tf.random.truncated_normal(dtype=tf.float64, shape=(1,), mean=0, stddev=0.01))
output = tf.map_fn(lambda h_t: tf.matmul(h_t, W_output) + b_output, gru.h_t)

# Create a placeholder for the expected output
expected_output = tf.compat.v1.placeholder(dtype=tf.float64, shape=(batch_size, None, 1), name='expected_output') ##Falls nicht geht None ersetzen

# Just use quadratic loss
loss = tf.reduce_sum(0.5 * tf.pow(output - expected_output, 2)) / float(batch_size)
##Loss könnte man noch anpassen

# Use the Adam optimizer for training
train_step = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

#train_step = tf.optimizers.Adam().minimize(loss, var_list=[W_output,b_output], tape=tf.GradientTape())


In [26]:

# Initialize all the variables
init_variables = tf.compat.v1.global_variables_initializer()
session.run(init_variables)

# Initialize the losses
train_losses = []
validation_losses = []
ep = []
epochs = 500

# Perform all the iterations
for epoch in range(epochs):
    # Compute the losses
    _, train_loss = session.run([train_step, loss], feed_dict={gru.input_layer: X_train, expected_output: Y_train})
    validation_loss = session.run(loss, feed_dict={gru.input_layer: X_test, expected_output: Y_test})

    # Log the losses
    #train_losses += [train_loss]
    #validation_losses += [validation_loss]

    # Display an update every 50 iterations
    if epoch % 50 == 0:
        ep += [epoch]
        train_losses += [train_loss]
        validation_losses += [validation_loss]
        #plt.plot(train_losses, '-b', label='Train loss')
        #plt.plot(validation_losses, '-r', label='Validation loss')
        #plt.legend(loc=0)
        #plt.title('Loss')
        #plt.xlabel('Iteration')
        #plt.ylabel('Loss')
        #plt.show()
        print('Iteration: %d, train loss: %.4f, test loss: %.4f' % (epoch, train_loss, validation_loss))

ValueError: setting an array element with a sequence.

In [ ]:
print(train_losses)

In [ ]:

# Define two numbers a and b and let the model compute a + b
#a = 1024
#b = 16

# The model is independent of the sequence length! Now we can test the model on even longer bitstrings
#bitstring_length = 20

# Create the feature vectors
#X_custom_sample = np.vstack([as_bytes(a, bitstring_length), as_bytes(b, bitstring_length)]).T
#X_custom = np.zeros((1,) + X_custom_sample.shape)
#X_custom[0, :, :] = X_custom_sample

# Make a prediction by using the model
#y_predicted = session.run(output, feed_dict={gru.input_layer: X_custom})
# Just use a linear class separator at 0.5
#y_bits = 1 * (y_predicted > 0.5)[0, :, 0]
# Join and reverse the bitstring
#y_bitstr = ''.join([str(int(bit)) for bit in y_bits.tolist()])[::-1]
# Convert the found bitstring to a number
#y = int(y_bitstr, 2)

# Print out the prediction
#print(y)  # Yay! This should equal 1024 + 16 = 1040